In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [3]:
train = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv"))

In [4]:
test = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv"))

In [5]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [6]:
train['Region_Code'] = train['Region_Code'].astype('int16')
train['Policy_Sales_Channel'] = train['Policy_Sales_Channel'].astype('int16')

In [10]:
cat_features = [0, 2, 3, 4, 5, 6, 8]

In [11]:
def strdataToInt(df):
    df['Vehicle_Damage']=df['Vehicle_Damage'].map({'Yes':1,'No':0})
    df['Gender']=df['Gender'].map({'Male':1,'Female':0})
    df['Vehicle_Age'] = df['Vehicle_Age'].map({'< 1 Year':1, '1-2 Year':2,'> 2 Years':3})
    return df

train = strdataToInt(train)
test = strdataToInt(test)

In [13]:
X = train.drop(['Response', 'id'], axis=1)
y = train['Response']

In [14]:
from imblearn.under_sampling import EditedNearestNeighbours

# EditedNearestNeighbours를 사용한 언더 샘플링
enn = EditedNearestNeighbours()
X, y = enn.fit_resample(X, y)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=22)

In [18]:
import gc
# 가비지 컬렉션 수행 전, 객체 수 확인
print("Before GC:", len(gc.get_objects()))

# 가비지 컬렉션 수행
gc.collect()

# 가비지 컬렉션 수행 후, 객체 수 확인
print("After GC:", len(gc.get_objects()))

Before GC: 223050
After GC: 222920


In [19]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=50,
    learning_rate=0.4,
    depth=8,
    one_hot_max_size=3,
    custom_loss=['AUC', 'Recall']
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
)

0:	learn: 0.4897563	test: 0.4897838	best: 0.4897838 (0)	total: 2.47s	remaining: 2m 1s
1:	learn: 0.3974693	test: 0.3975683	best: 0.3975683 (1)	total: 4.44s	remaining: 1m 46s
2:	learn: 0.3463928	test: 0.3465061	best: 0.3465061 (2)	total: 6.8s	remaining: 1m 46s
3:	learn: 0.3162136	test: 0.3163522	best: 0.3163522 (3)	total: 8.73s	remaining: 1m 40s
4:	learn: 0.2975776	test: 0.2977112	best: 0.2977112 (4)	total: 10.8s	remaining: 1m 36s
5:	learn: 0.2845493	test: 0.2847243	best: 0.2847243 (5)	total: 13s	remaining: 1m 35s
6:	learn: 0.2736979	test: 0.2738839	best: 0.2738839 (6)	total: 15.3s	remaining: 1m 33s
7:	learn: 0.2674733	test: 0.2676686	best: 0.2676686 (7)	total: 17.5s	remaining: 1m 31s
8:	learn: 0.2634556	test: 0.2636419	best: 0.2636419 (8)	total: 19.4s	remaining: 1m 28s
9:	learn: 0.2598882	test: 0.2600414	best: 0.2600414 (9)	total: 21.4s	remaining: 1m 25s
10:	learn: 0.2576061	test: 0.2577330	best: 0.2577330 (10)	total: 23.6s	remaining: 1m 23s
11:	learn: 0.2562519	test: 0.2563739	best: 0.

In [20]:
y_pred = clf.predict(data=X_val)

In [21]:
def calc_accuracy(y_pred, y_val):
    acc = y_val == y_pred
    i = 0
    for d in acc:
        if d == True:
            i+=1
    s, n = 0, 0
    for j in y_pred:
        n+=1
        if j==1:
            s += 1
    print(i/acc.size, s, n)

In [22]:
calc_accuracy(y_pred, y_val)

0.8750943514205193 112260 895588


In [23]:
X_all = train.drop(['Response', 'id'], axis=1)
y_pred_all = clf.predict(data=X_all)

In [24]:
#please execute once
y_all = train['Response']

In [25]:
calc_accuracy(y_pred_all, y_all)

0.8477866364972249 1757111 11504798


In [20]:
test['Region_Code'] = test['Region_Code'].astype('int16')
test['Policy_Sales_Channel'] = test['Policy_Sales_Channel'].astype('int16')

In [21]:
X_sub = test.drop('id', axis=1)

In [22]:
y_sub_pred = clf.predict(data=X_sub)

In [23]:
submission = pd.DataFrame(pd.read_csv("/kaggle/input/playground-series-s4e7/sample_submission.csv"))

In [24]:
submission['Response'] = y_sub_pred

In [26]:
print(submission.head())
print(submission.describe())

         id  Response
0  11504798         0
1  11504799         1
2  11504800         0
3  11504801         0
4  11504802         0
                 id      Response
count  7.669866e+06  7.669866e+06
mean   1.533973e+07  1.293667e-01
std    2.214100e+06  3.356053e-01
min    1.150480e+07  0.000000e+00
25%    1.342226e+07  0.000000e+00
50%    1.533973e+07  0.000000e+00
75%    1.725720e+07  0.000000e+00
max    1.917466e+07  1.000000e+00


In [27]:
submission.to_csv('submission.csv' , index = False)